### Zaawansowana klasyfikacja

Wykorzystajmy zestaw danych, który ma trochę bardziej złożone cechy - zarówno ciągłe jak i kategoryczne. Jest to zestaw danych bazujący na spisie ludności. Podane są różne dane socjometryczne i jedna zmienna binarna - czy przychody są większe niż $50K.

Plik: `income_evaluation.csv`
Więcej na: https://www.kaggle.com/uciml/adult-census-income

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read data sets - https://docs.python.org/3/library/re.html
df = pd.read_csv("./data/income_evaluation.csv", sep='\s*,\s*', header=0, encoding='ascii', engine='python')
df.head()

In [ ]:
df.income.value_counts()

In [ ]:
?pd.read_csv

In [ ]:
# Czy "fnlwgt" przyczyni się do poprawy jakości modelu
#fnlwgt - https://www.kaggle.com/uciml/adult-census-income/discussion/32698

df.fnlwgt.count(), df.fnlwgt.nunique()

In [ ]:
df.info()

b. Konwersja danych kategorycznych do numerycznych 
`sklearn.preprocessing.LabelEncoder`

In [ ]:
#encoding categorical to numeric
from sklearn.preprocessing import LabelEncoder

list_to_encode = [
    'workclass', 'education', 'marital-status', 'occupation', 
    'relationship', 'race', 'sex', 'native-country'
]

df_encoded = pd.get_dummies(df, columns=list_to_encode)
df_encoded.head()

In [ ]:
list(df_encoded.columns)

c. Wybór cech oraz wyjścia modelu

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(df_encoded['income'])
x = df_encoded.drop(['income'], axis=1)

In [ ]:
sum(y)/len(y)

`xgboost` zawiera specyficzną strukturę danych, której używa wewnętrznie do treningu. Jest ona zoptymalizowana pod wykorzystanie w tym pakiecie.

In [ ]:
data_dm = xgb.DMatrix(data=x.values, label=y)

train - przygotowujemy model               ------ cv


validacyjny - dostrajamy parametry          ------ cv

-----------------
testowy - finalny test

In [ ]:
data_dm

Użyjmy crosswalidacji. Metoda `xgb.cv` działa bardzo podobnie do analogicznych metod z pakietu `sklearn.model_selection` (`cross_val_score`).

In [ ]:
xgb.cv?

In [ ]:
# auc == AUROC == pole pod krzywą ROC

params = {
    "objective": "reg:logistic", 
    "max_depth": 3, 
    "booster": "gbtree",
    "learning_rate": 0.8  # DEFAULT: 0.3
}


cv_results = xgb.cv(
    dtrain=data_dm, 
    params=params, 
    nfold=4, 
    num_boost_round=250, metrics=["error", "auc"], as_pandas=True) #metrics="auc"


cv_results

In [ ]:
cv_results.columns

In [ ]:
plt.figure(figsize=(15, 7))
plt.title("Błąd na danych treningowych i testowych")
plt.plot(cv_results["train-error-mean"], color="b")
plt.plot(cv_results["test-error-mean"], color="r")
# plt.ylim((0., 0.2))
plt.hlines([cv_results["test-error-mean"].min()], xmin=0, xmax=250, color="g")
plt.xlabel("Ilość klasyfikatorów")
plt.ylabel("Wartość funkcji błędu")

In [ ]:
plt.figure(figsize=(15, 7))
plt.title("AUC na danych treningowych i testowych")
plt.plot(cv_results["train-auc-mean"], color="b")
plt.plot(cv_results["test-auc-mean"], color="r")
# plt.ylim((0., 0.2))
plt.xlabel("Ilość klasyfikatorów")
plt.ylabel("Wartość auc")

In [ ]:
# Spróbujmy użyć klasycznego model_selection - czy wyniki są mniej więcej zgodne?

import sklearn.model_selection as msel

# .score() - accuracy

clf = xgb.XGBClassifier(objective="reg:logistic", max_depth=3, 
                        learning_rate=0.8, n_estimators=5)
msel.cross_val_score(clf, x, y, cv=4, n_jobs=-1)

Model selection ocenia dokładność, podczas gdy xgb.cv ocenia błąd. Błąd = 1 - dokładność.

In [ ]:
# results show
print("cv_Results: ", cv_results.iloc[-1])

h. przedstawienie accuracy oraz auc

In [ ]:
#accuracy/average out-of-sample accuracy
print("acc: ", ((1- cv_results["test-error-mean"]).iloc[-1]))

### Zadanie 3

Zmodyfikuj kod powyżej tak, aby wykorzystywać sposób w jaki powinno się robić crosswalidację: użyj zestawu treningowego do walidacji (zostanie on wewnętrznie podzielony na zestaw treningowy i walidacyjny) oraz małego zestawu testowego do oceny dokładności wybranego najlepszego modelu.

Uzasadnienie: taką crosswalidację będziemy chcieli wykonać wielokrotnie w ramach poszukiwania parametrów. Zachowajmy zestaw testowy do oceny ostatecznego rozwiązania.

Wykorzystaj metody poznane wcześniej na kursie.

Spróbujmy trzech kombinacji parametrów.

In [ ]:
# 0. Podział na trening_valid, test
# 1. Trening na kilku wariantach i ocena (CV)  [max_depth=2,3,4, learning_rate=0.8, 0.5]
# 2. Na najlepszym modelu, trening na trening_valid a test na test